Create_contact_info

In [16]:
import os
import numpy as np
import h5py
import glob
import argparse
import sys

# ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(ROOT_DIR)

from contact_graspnet.data import PointCloudReader
from contact_graspnet.mesh_utils import in_collision_with_gripper, grasp_contact_location

import pprint as pp 


In [18]:
# root_folder = '/home/tele_mani_u20/code_proj/cgn/June_4/contact_graspnet/acronym_generate_scene/'
root_folder = '/home/tele_mani_u20/code_proj/cgn/June_4/contact_graspnet/acronym/'

pcreader = PointCloudReader(root_folder=root_folder)

grasp_paths = glob.glob(os.path.join(root_folder, 'grasps', '*.h5'))
    
print('Computing grasp contacts...')
for grasp_path in grasp_paths:
    # print('Reading: ', grasp_path)

    target_path = os.path.join(pcreader._root_folder, 'mesh_contacts')
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    
    # 'acronym_generate_scene/mesh_contacts/Gun_fe62130ce6fcd9b77754fed890b42399_0.015994739372013036.npz'
    contact_dir_path = os.path.join(target_path, os.path.basename(grasp_path.replace('.h5','.npz')))
    if not os.path.exists(os.path.dirname(contact_dir_path)):
        os.makedirs(os.path.dirname(contact_dir_path))
    if os.path.exists(contact_dir_path):
        print('path exists', contact_dir_path)

    # Begin function read_object_grasp_data_acronym
    h5_path = grasp_path

    abs_h5_path = os.path.join(root_folder, 'grasps', h5_path)
    data = h5py.File(abs_h5_path, "r")

    # /home/tele_mani_u20/code_proj/cgn/June_4/contact_graspnet/acronym_generate_scene/meshes/Gun/fe62130ce6fcd9b77754fed890b42399.obj
    print(data["object/file"][()])
    mesh_fname = 'meshes/' + data["object/file"][()].split('/')[-1]
    mesh_fname = os.path.join(root_folder, mesh_fname)#.decode('utf-8')

    mesh_scale = data["object/scale"][()]
    
    # How to interpret the frame these grasps are represented wrt
    # https://github.com/NVlabs/acronym/blob/main/scripts/acronym_visualize_grasps.py
    # (2000, 4, 4) 
    grasps = np.array(data["grasps/transforms"])
    
    # (2000,)
    success = np.array(data["grasps/qualities/flex/object_in_gripper"])

    positive_grasps = grasps[success==1, :, :]
    negative_grasps = grasps[success==0, :, :]

    print('positive grasps: {} negative grasps: {}'.format(positive_grasps.shape, negative_grasps.shape))

    output_grasps, output_labels, cad_path, cad_scale = grasps, success, mesh_fname, mesh_scale
    # End # Begin function read_object_grasp_data_acronym

    # This fn will also shift the mesh vertices
    # such that the mean of the mesh is at origin
    pcreader._renderer._load_object(cad_path, cad_scale)

    context = pcreader._renderer._cache[(cad_path,cad_scale)]
    obj_mesh = context['tmesh']
    obj_mesh_mean = context['mesh_mean']

    print('obj_mesh_mean', obj_mesh_mean)

    output_grasps[:,:3,3] -= obj_mesh_mean

    # Begin grasps_contact_info(output_grasps, list(output_labels), obj_mesh, check_collisions=False)
    grasp_tfs, successfuls, obj_mesh = output_grasps, list(output_labels), obj_mesh

    # Begin grasp_contact_location(
    #     grasp_tfs,
    #     successfuls,
    #     collisions if check_collisions else [0]*len(successfuls),
    #     object_mesh=obj_mesh,
    #     gripper_name='panda',
    #     silent=True,
    # )
    transforms, successfuls = grasp_tfs, successfuls
    collisions = [0]*len(successfuls)   
    object_mesh = obj_mesh
    gripper_name = 'panda' 
    silent = False

    from contact_graspnet.mesh_utils import create_gripper
    import trimesh 
    res = []
    gripper = create_gripper(gripper_name)

    intersector = trimesh.ray.ray_triangle.RayMeshIntersector(object_mesh)

    for p, colliding, outcome in zip(transforms, collisions, successfuls):

        contact_dict = {}
        contact_dict['collisions'] = 0
        contact_dict['valid_locations'] = 0
        contact_dict['successful'] = outcome
        contact_dict['grasp_transform'] = p
        contact_dict['contact_points'] = []
        contact_dict['contact_directions'] = []
        contact_dict['contact_face_normals'] = []
        contact_dict['contact_offsets'] = []

        assert not colliding

        # [[ 0.0399  0.      0.1034  1.    ]
        #  [-0.0399  0.      0.1034  1.    ]]
        # print(gripper.contact_ray_origins)

        # [[-1.  0.  0.]
        #  [ 1. -0. -0.]]
        # print(gripper.contact_ray_directions)

        ray_origins, ray_directions = gripper.get_closing_rays_contact(p)

        # https://trimsh.org/examples/ray.html
        # [[-0.04373895 -0.00576017 -0.04461852]
        #  [-0.04929868  0.00563721 -0.04436079]] 
        # [0 1] 
        # [ 1192 19008]
        locations, index_ray, index_tri = intersector.intersects_location(
            ray_origins, ray_directions, multiple_hits=False)

        if len(locations) > 0:
            # this depends on the width of the gripper
            # [ True  True]
            valid_locations = np.linalg.norm(ray_origins[index_ray]-locations, axis=1) <= 2.0*gripper.q

            if sum(valid_locations) > 1:
                contact_dict['valid_locations'] = 1
                contact_dict['contact_points'] = locations[valid_locations]
                contact_dict['contact_face_normals'] = object_mesh.face_normals[index_tri[valid_locations]]
                contact_dict['contact_directions'] = ray_directions[index_ray[valid_locations]]
                contact_dict['contact_offsets'] = np.linalg.norm(ray_origins[index_ray[valid_locations]] - locations[valid_locations], axis=1)
                # dot_prods = (contact_dict['contact_face_normals'] * contact_dict['contact_directions']).sum(axis=1)
                # contact_dict['contact_cosine_angles'] = np.cos(dot_prods)

                # {'collisions': 0,
                #  'contact_directions': array([[-0.43833588,  0.89858154,  0.02031917],
                #        [ 0.43833588, -0.89858154, -0.02031917]]),
                #  'contact_face_normals': array([[ 0., -1.,  0.],
                #        [-0.,  1.,  0.]]),
                #  'contact_offsets': array([0.03208667, 0.03502959]),
                #  'contact_points': array([[-0.04373895, -0.00576017, -0.04461852],
                #        [-0.04929868,  0.00563721, -0.04436079]]),
                #  'grasp_transform': array([[ 0.43833588, -0.46688769,  0.76803486, -0.12657861],
                #        [-0.89858154, -0.20831802,  0.38620566, -0.03867292],
                #        [-0.02031917, -0.85942975, -0.51084992,  0.00836213],
                #        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
                #  'successful': 1,
                #  'valid_locations': 1}
                # pp.pprint(contact_dict)
                
                res.append(contact_dict)

    contact_dicts = res 

    contact_dict_of_arrays = {}
    for d in contact_dicts:
        for k in d:
            contact_dict_of_arrays.setdefault(k,[]).append(d[k])

    # This saves it to 
    # acronym_generate_scene/mesh_contacts/Gun_fe62130ce6fcd9b77754fed890b42399_0.015994739372013036.npz
    np.savez(contact_dir_path, **contact_dict_of_arrays)

    break

********** terminating renderer **************
Computing grasp contacts...
path exists /home/tele_mani_u20/code_proj/cgn/June_4/contact_graspnet/acronym/mesh_contacts/Gun_fe62130ce6fcd9b77754fed890b42399_0.015994739372013036.npz
meshes/Gun/fe62130ce6fcd9b77754fed890b42399.obj
positive grasps: (1754, 4, 4) negative grasps: (246, 4, 4)
obj_mesh_mean [[ 0.49780753 -0.02607832  0.93002697]]
